In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, classification_report
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from transformers import AdamW
from tqdm import tqdm

In [22]:
pip install fugashi ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 42.9 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=6702ab3a4b38be4540fc607ae090b9326e60c4b3d1d3fa08f2e17db8ba36408a
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic


In [46]:
language="jp"
modelname="JapaneseBERT"
input = "OPENMWE"
test = "OPENMWE"

In [47]:
if input == "gpt4":
  gpt4_flag=True
else:
  gpt4_flag=False

In [48]:
# JP encoding='euc-jp' other encoding normal CHECK ALWAYS DATASETS
df_train = pd.read_csv(f'jp_train.csv',encoding='euc-jp')
df_eval = pd.read_csv(f'jp_eval.csv',encoding='euc-jp')
df_test = pd.read_csv(f'jp_test.csv',encoding='euc-jp')

In [49]:
def get_category_mapping(language):
    if language in ['en', 'jp']:
        category_mapping = {
            'figurative': 'idiom',
            'literal': 'nonidiom'
        }
    elif language == 'tr':
        category_mapping = {
            'mecaz': 'idiom',
            'gerçek': 'nonidiom'
        }
    elif language == 'it':
        category_mapping = {
            'figurato': 'idiom',
            'letterale': 'nonidiom'
        }
    else:
        raise ValueError(f"Language '{language}' is not supported.")

    return category_mapping

In [50]:
category_mapping = get_category_mapping(language)
print(category_mapping)

{'figurative': 'idiom', 'literal': 'nonidiom'}


In [51]:
if gpt4_flag or language in ['en', 'jp']:
  df_train['category'] = df_train['category'].map(category_mapping)
df_train

,category,id,idiom,submission
0,idiom,35,あぐらをかく,過去に<あぐらを><かくのではなく、>世界の中でも大きなマーケットである日本を自覚し、開かれ...
1,idiom,35,あぐらをかく,娘の亭主にずけずけとものを言う藤村志保や、鈍感すれすれの神経の太さを見せる富司純子、確信犯的...
2,idiom,35,あぐらをかく,いまや多くの国民・労働者が自民党をぶっつぶす内閣ではなくて、それどころか利権と金権、腐敗の上...
3,idiom,35,あぐらをかく,そのような言葉が生まれた経緯は詳しく知りませんが、役所が長い年月を経て強大な権限を持つように...
4,idiom,35,あぐらをかく,プロ野球界からこのような成果がほとんど聞かれないのは プロ野球に携わる球団関係者、協会が人気...
...,...,...,...,...
30305,nonidiom,2621,鼻を鳴らす,特徴は語尾がものすごく上がり「・・・くしょい」のあとクンクンクンっと<鼻を><鳴らすのが>彼...
30306,nonidiom,2621,鼻を鳴らす,スカートを更にまくり、白いヒップを生で眼前に突きだしてやると、浩一は<鼻を>クンクン<鳴らし...
30307,nonidiom,2621,鼻を鳴らす,それぞれのスケルトンの出来を確かめるように、上から下までしげしげと見つめながら女性は<鼻を>...
30308,nonidiom,2621,鼻を鳴らす,だが、馬たちの走り回る向こうに何かを見つけて、ぴくりと耳を動かし、<鼻を><鳴らして、>スェ...


In [52]:
if gpt4_flag or language in ['en', 'jp']:
  df_eval['category'] = df_eval['category'].map(category_mapping)
df_eval

,category,id,idiom,submission
0,idiom,35,あぐらをかく,身分安泰として、官僚主義に<あぐらを><かく>人物が采配しているとすれば、自治体制度そのもの...
1,idiom,35,あぐらをかく,相手が遠慮したり、気を遣ってくれたりする「ラクさ」の上に<あぐらを><かいているような>男は...
2,idiom,35,あぐらをかく,そんな事情からか、利用者の「工事中では仕方がない」という酌量に<あぐらを><かいて、>不自由...
3,idiom,35,あぐらをかく,今までのキャリアの上に<あぐらを><かいた>焼き直しではなく、確実に今現在の彼の等身大のサウ...
4,idiom,35,あぐらをかく,「戦時中、俺は自らの生命を賭けて、国の為に戦った。俺達大人は、ひもじい思いをしながらも、飛び...
...,...,...,...,...
5114,nonidiom,2621,鼻を鳴らす,セントバーナードは最後にぶしゅん、と<鼻を><鳴らすと、>前足を若菜の腰のあたりに置いた。
5115,nonidiom,2621,鼻を鳴らす,快感に<鼻を><鳴らしても、>ドロシーの唇はミンミの秘所から離れる事は無く、舌を膣内で蠢かせる。
5116,nonidiom,2621,鼻を鳴らす,わずかにぐずぐずと<鼻を><鳴らしながら、>自分を非難する少年の、その言い様ももっともと、彼...
5117,nonidiom,2621,鼻を鳴らす,とっても甘えん坊のらんらんちゃん、眠くなると「抱っこして~」と言いたげに「くんくん」と<鼻を...


In [53]:
if language in ['en', 'jp']:
  df_test['category'] = df_test['category'].map(category_mapping)

In [54]:
df_test

,category,id,idiom,submission
0,idiom,35,あぐらをかく,この財政赤字の時代に民間だけが運営・税務対策・価格競争を生き抜いて戦っている環境の中で、何も...
1,idiom,35,あぐらをかく,教育改革云々という言葉が連日、新聞紙面やテレビを賑わしていますが、本校も百年近い歴史に<あぐ...
2,idiom,35,あぐらをかく,ソフトウェアの利用者は、それらの工夫の為に、そのソフトウェアが問題を抱えていると自覚する機会...
3,idiom,35,あぐらをかく,自然を大切に暮らしてきた人たちの心の暖かさや、決して伝統の上に<あぐらを><かくことなく>歴...
4,idiom,35,あぐらをかく,明治１６年創業の同店は、「のれん」に<あぐらを><かくことなく、>きものが好きな方々に少しで...
...,...,...,...,...
15234,nonidiom,2621,鼻を鳴らす,「なぜ、この子を殴る！？ 今しがたの件に、何の関係があったというのだ！！」 男はフンと<鼻を...
15235,nonidiom,2621,鼻を鳴らす,「ガルア、もういいわ……離しなさい……」 リリスとシェシェルに少し離れて桃園市の上空を飛んで...
15236,nonidiom,2621,鼻を鳴らす,ぐす、と<鼻を><鳴らす>敏生に、森はこの少年に感じさせてしまった孤独感に胸が苦しくなった。
15237,nonidiom,2621,鼻を鳴らす,ある時には，ハミングするような音を出したり，<鼻を>クンクン<鳴らしたり，また，>ある時には...


In [55]:
df_train['category'] = df_train['category'].str.lower()
df_eval['category'] = df_eval['category'].str.lower()
df_test['category'] = df_test['category'].str.lower()

In [56]:
df_train.head()

,category,id,idiom,submission
0,idiom,35,あぐらをかく,過去に<あぐらを><かくのではなく、>世界の中でも大きなマーケットである日本を自覚し、開かれ...
1,idiom,35,あぐらをかく,娘の亭主にずけずけとものを言う藤村志保や、鈍感すれすれの神経の太さを見せる富司純子、確信犯的...
2,idiom,35,あぐらをかく,いまや多くの国民・労働者が自民党をぶっつぶす内閣ではなくて、それどころか利権と金権、腐敗の上...
3,idiom,35,あぐらをかく,そのような言葉が生まれた経緯は詳しく知りませんが、役所が長い年月を経て強大な権限を持つように...
4,idiom,35,あぐらをかく,プロ野球界からこのような成果がほとんど聞かれないのは プロ野球に携わる球団関係者、協会が人気...


In [57]:
df_eval.head()

,category,id,idiom,submission
0,idiom,35,あぐらをかく,身分安泰として、官僚主義に<あぐらを><かく>人物が采配しているとすれば、自治体制度そのもの...
1,idiom,35,あぐらをかく,相手が遠慮したり、気を遣ってくれたりする「ラクさ」の上に<あぐらを><かいているような>男は...
2,idiom,35,あぐらをかく,そんな事情からか、利用者の「工事中では仕方がない」という酌量に<あぐらを><かいて、>不自由...
3,idiom,35,あぐらをかく,今までのキャリアの上に<あぐらを><かいた>焼き直しではなく、確実に今現在の彼の等身大のサウ...
4,idiom,35,あぐらをかく,「戦時中、俺は自らの生命を賭けて、国の為に戦った。俺達大人は、ひもじい思いをしながらも、飛び...


In [58]:
df_test.head()

,category,id,idiom,submission
0,idiom,35,あぐらをかく,この財政赤字の時代に民間だけが運営・税務対策・価格競争を生き抜いて戦っている環境の中で、何も...
1,idiom,35,あぐらをかく,教育改革云々という言葉が連日、新聞紙面やテレビを賑わしていますが、本校も百年近い歴史に<あぐ...
2,idiom,35,あぐらをかく,ソフトウェアの利用者は、それらの工夫の為に、そのソフトウェアが問題を抱えていると自覚する機会...
3,idiom,35,あぐらをかく,自然を大切に暮らしてきた人たちの心の暖かさや、決して伝統の上に<あぐらを><かくことなく>歴...
4,idiom,35,あぐらをかく,明治１６年創業の同店は、「のれん」に<あぐらを><かくことなく、>きものが好きな方々に少しで...


In [59]:
# Encode the 'category' column
label_encoder = LabelEncoder()
df_train['category_encoded'] = label_encoder.fit_transform(df_train['category'])
df_eval['category_encoded'] = label_encoder.transform(df_eval['category'])
df_test['category_encoded'] = label_encoder.transform(df_test['category'])

In [60]:
df_train

,category,id,idiom,submission,category_encoded
0,idiom,35,あぐらをかく,過去に<あぐらを><かくのではなく、>世界の中でも大きなマーケットである日本を自覚し、開かれ...,0
1,idiom,35,あぐらをかく,娘の亭主にずけずけとものを言う藤村志保や、鈍感すれすれの神経の太さを見せる富司純子、確信犯的...,0
2,idiom,35,あぐらをかく,いまや多くの国民・労働者が自民党をぶっつぶす内閣ではなくて、それどころか利権と金権、腐敗の上...,0
3,idiom,35,あぐらをかく,そのような言葉が生まれた経緯は詳しく知りませんが、役所が長い年月を経て強大な権限を持つように...,0
4,idiom,35,あぐらをかく,プロ野球界からこのような成果がほとんど聞かれないのは プロ野球に携わる球団関係者、協会が人気...,0
...,...,...,...,...,...
30305,nonidiom,2621,鼻を鳴らす,特徴は語尾がものすごく上がり「・・・くしょい」のあとクンクンクンっと<鼻を><鳴らすのが>彼...,1
30306,nonidiom,2621,鼻を鳴らす,スカートを更にまくり、白いヒップを生で眼前に突きだしてやると、浩一は<鼻を>クンクン<鳴らし...,1
30307,nonidiom,2621,鼻を鳴らす,それぞれのスケルトンの出来を確かめるように、上から下までしげしげと見つめながら女性は<鼻を>...,1
30308,nonidiom,2621,鼻を鳴らす,だが、馬たちの走り回る向こうに何かを見つけて、ぴくりと耳を動かし、<鼻を><鳴らして、>スェ...,1


In [61]:
df_eval

,category,id,idiom,submission,category_encoded
0,idiom,35,あぐらをかく,身分安泰として、官僚主義に<あぐらを><かく>人物が采配しているとすれば、自治体制度そのもの...,0
1,idiom,35,あぐらをかく,相手が遠慮したり、気を遣ってくれたりする「ラクさ」の上に<あぐらを><かいているような>男は...,0
2,idiom,35,あぐらをかく,そんな事情からか、利用者の「工事中では仕方がない」という酌量に<あぐらを><かいて、>不自由...,0
3,idiom,35,あぐらをかく,今までのキャリアの上に<あぐらを><かいた>焼き直しではなく、確実に今現在の彼の等身大のサウ...,0
4,idiom,35,あぐらをかく,「戦時中、俺は自らの生命を賭けて、国の為に戦った。俺達大人は、ひもじい思いをしながらも、飛び...,0
...,...,...,...,...,...
5114,nonidiom,2621,鼻を鳴らす,セントバーナードは最後にぶしゅん、と<鼻を><鳴らすと、>前足を若菜の腰のあたりに置いた。,1
5115,nonidiom,2621,鼻を鳴らす,快感に<鼻を><鳴らしても、>ドロシーの唇はミンミの秘所から離れる事は無く、舌を膣内で蠢かせる。,1
5116,nonidiom,2621,鼻を鳴らす,わずかにぐずぐずと<鼻を><鳴らしながら、>自分を非難する少年の、その言い様ももっともと、彼...,1
5117,nonidiom,2621,鼻を鳴らす,とっても甘えん坊のらんらんちゃん、眠くなると「抱っこして~」と言いたげに「くんくん」と<鼻を...,1


In [62]:
df_test

,category,id,idiom,submission,category_encoded
0,idiom,35,あぐらをかく,この財政赤字の時代に民間だけが運営・税務対策・価格競争を生き抜いて戦っている環境の中で、何も...,0
1,idiom,35,あぐらをかく,教育改革云々という言葉が連日、新聞紙面やテレビを賑わしていますが、本校も百年近い歴史に<あぐ...,0
2,idiom,35,あぐらをかく,ソフトウェアの利用者は、それらの工夫の為に、そのソフトウェアが問題を抱えていると自覚する機会...,0
3,idiom,35,あぐらをかく,自然を大切に暮らしてきた人たちの心の暖かさや、決して伝統の上に<あぐらを><かくことなく>歴...,0
4,idiom,35,あぐらをかく,明治１６年創業の同店は、「のれん」に<あぐらを><かくことなく、>きものが好きな方々に少しで...,0
...,...,...,...,...,...
15234,nonidiom,2621,鼻を鳴らす,「なぜ、この子を殴る！？ 今しがたの件に、何の関係があったというのだ！！」 男はフンと<鼻を...,1
15235,nonidiom,2621,鼻を鳴らす,「ガルア、もういいわ……離しなさい……」 リリスとシェシェルに少し離れて桃園市の上空を飛んで...,1
15236,nonidiom,2621,鼻を鳴らす,ぐす、と<鼻を><鳴らす>敏生に、森はこの少年に感じさせてしまった孤独感に胸が苦しくなった。,1
15237,nonidiom,2621,鼻を鳴らす,ある時には，ハミングするような音を出したり，<鼻を>クンクン<鳴らしたり，また，>ある時には...,1


In [63]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [64]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("tohoku-nlp/bert-base-japanese")

# Tokenize the data
train_encodings = tokenizer(list(df_train['submission']), truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(list(df_eval['submission']), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(list(df_test['submission']), truncation=True, padding=True, return_tensors="pt")
# Convert to torch tensors
train_labels = torch.tensor(df_train['category_encoded'].values)
val_labels = torch.tensor(df_eval['category_encoded'].values)
test_labels = torch.tensor(df_test['category_encoded'].values)

# Prepare to datasets
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained("tohoku-nlp/bert-base-japanese", num_labels=num_labels)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-6)

epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Average loss across all batches
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Evaluation step
    model.eval()
    total_eval_accuracy = 0
    for batch in tqdm(val_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        accuracy = (predictions == batch['labels']).cpu().numpy().mean() * 100
        total_eval_accuracy += accuracy

    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Validation Accuracy: {avg_val_accuracy:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/3789 [00:00<?, ?it/s]<ipython-input-63-845b27163e62>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-63-845b27163e62>:8: Us

In [ ]:
# Ensure the model is in evaluation mode
model.eval()

# Move the model to the appropriate device
model.to(device)

# Initialize variables to track accuracy
total_correct = 0
total_predictions = 0

# Store true labels and predictions for F1 score calculation
all_labels = []
all_predictions = []

# No gradient updates needed for evaluation
with torch.no_grad():
    for batch in test_loader:
        # Move batch to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Get predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Update tracking variables
        total_correct += (predictions == batch['labels']).sum().item()
        total_predictions += predictions.size(0)

        # Store predictions and true labels
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Calculate accuracy
test_accuracy = total_correct / total_predictions
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate F1 scores
f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
f1_macro = f1_score(all_labels, all_predictions, average='macro')

print(f'Weighted F1 Score: {f1_weighted:.4f}')
print(f'Macro F1 Score: {f1_macro:.4f}')


In [ ]:
# Generate classification report
report = classification_report(all_labels, all_predictions, digits=4)
print(f'Classification Report for {modelname}, language {language.upper()}, Train {input.upper()}, Test {test.upper()}:')
print(report)

In [45]:
# Generate classification report
report = classification_report(all_labels, all_predictions, digits=4)
print(f'Classification Report for {modelname}, language {language.upper()}, Train {input.upper()}, Test {test.upper()}:')
print(report)

Classification Report for JapaneseBERT, language JP, Train GPT4, Test OPENMWE:
              precision    recall  f1-score   support

           0     0.8826    0.8951    0.8888     10781
           1     0.7373    0.7120    0.7244      4458

    accuracy                         0.8415     15239
   macro avg     0.8099    0.8035    0.8066     15239
weighted avg     0.8401    0.8415    0.8407     15239

